In [ ]:
import torch
import random
import calendar

import platform
import sys

# Проверка доступности GPU (как мы делали раньше)
# Если CUDA есть, используем её. Если нет - процессор.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"torch.version.cuda: {torch.version.cuda}")       # Должно показать '12.4' или '12.1' (не None!)
print(f"torch.cuda.is_available(): {torch.cuda.is_available()}") # Должно быть True
print(f"torch.xpu.is_available(): {torch.xpu.is_available()}") 

print(f"✅ Устройство для вычислений: {device}")

torch.version.cuda: 12.8
torch.cuda.is_available(): False
✅ Устройство для вычислений: cpu


In [2]:
print(f"sys.version: {sys.version}")

print(f"platform.python_version(): {platform.python_version()}")

sys.version: 3.12.3 (main, Nov  6 2025, 13:44:16) [GCC 13.3.0]
platform.python_version(): 3.12.3


In [3]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug',
          'Sep', 'Oct', 'Nov', 'Dec']


def generate_date():
    year = random.randint(1950, 2050)

    monthIdx = random.randint(1, 12)

    match monthIdx:
        case 1 | 3 | 5 | 7 | 8 | 10 | 12:
            day = random.randint(1, 31)
        case 4 | 6 | 9 | 11:
            day = random.randint(1, 30)
        case 2:
            if calendar.isleap(year):
                day = random.randint(1, 29)
            else:
                day = random.randint(1, 28)

    # 1. Порядок: Год - Месяц - День
    # 2. Форматирование :02d добавляет ноль в начале, если число < 10 (например, 05)
    tgt_str = f'{year}-{monthIdx:02d}-{day:02d}'

    # Вход оставляем как есть: "25 Jan 2023"
    src_str = f'{day} {months[monthIdx-1]} {year}'

    # Добавляем маркеры старта и стопа
    tgt_str = f'<{tgt_str}>'

    return src_str, tgt_str


print(generate_date())

('28 Oct 1976', '<1976-10-28>')


In [4]:
# 1. Определяем все возможные символы (Alphabet)
# set() убирает дубликаты
chars = set("0123456789 abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ<>-")


# 2. Создаем Map: Символ -> Индекс (Integer)
# sorted() сортирует, чтобы порядок всегда был одинаковым
# enumerate() возвращает пары (счетчик, элемент)
char2idx = {char: idx for idx, char in enumerate(sorted(chars))}

print(f"char2idx: {char2idx}")

# 3. Создаем обратный Map: Индекс -> Символ (для расшифровки ответа)
idx2char = {idx: char for char, idx in char2idx.items()}

print(f"idx2char: {idx2char}")


# Добавляем служебный токен "EOS" (End Of Sentence) или PAD, если нужно
# Пока просто запомним размер нашего "алфавита"
vocab_size = len(char2idx)

print(f"Размер словаря (количество уникальных символов): {vocab_size}")
print(f"Индекс буквы 'A': {char2idx['A']}")
print(f"Символ под индексом 10: {idx2char[10]}")

char2idx: {' ': 0, '-': 1, '0': 2, '1': 3, '2': 4, '3': 5, '4': 6, '5': 7, '6': 8, '7': 9, '8': 10, '9': 11, '<': 12, '>': 13, 'A': 14, 'B': 15, 'C': 16, 'D': 17, 'E': 18, 'F': 19, 'G': 20, 'H': 21, 'I': 22, 'J': 23, 'K': 24, 'L': 25, 'M': 26, 'N': 27, 'O': 28, 'P': 29, 'Q': 30, 'R': 31, 'S': 32, 'T': 33, 'U': 34, 'V': 35, 'W': 36, 'X': 37, 'Y': 38, 'Z': 39, 'a': 40, 'b': 41, 'c': 42, 'd': 43, 'e': 44, 'f': 45, 'g': 46, 'h': 47, 'i': 48, 'j': 49, 'k': 50, 'l': 51, 'm': 52, 'n': 53, 'o': 54, 'p': 55, 'q': 56, 'r': 57, 's': 58, 't': 59, 'u': 60, 'v': 61, 'w': 62, 'x': 63, 'y': 64, 'z': 65}
idx2char: {0: ' ', 1: '-', 2: '0', 3: '1', 4: '2', 5: '3', 6: '4', 7: '5', 8: '6', 9: '7', 10: '8', 11: '9', 12: '<', 13: '>', 14: 'A', 15: 'B', 16: 'C', 17: 'D', 18: 'E', 19: 'F', 20: 'G', 21: 'H', 22: 'I', 23: 'J', 24: 'K', 25: 'L', 26: 'M', 27: 'N', 28: 'O', 29: 'P', 30: 'Q', 31: 'R', 32: 'S', 33: 'T', 34: 'U', 35: 'V', 36: 'W', 37: 'X', 38: 'Y', 39: 'Z', 40: 'a', 41: 'b', 42: 'c', 43: 'd', 44: 'e',

In [5]:
import torch
import torch.nn as nn


class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # 1. Слой Embeddings
        # Это словарь, который превращает индекс символа (int) в вектор (float[]).
        # input_size: размер словаря (наш vocab_size, около 66 символов)
        # hidden_size: размер вектора (например, 128 чисел на одну букву)
        self.embedding = nn.Embedding(input_size, hidden_size)

        # 2. Слой GRU (Gated Recurrent Unit)
        # Это сама рекуррентная "память". 
        # Она берет вектор буквы и старую память -> выдает новую память.
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        # input: это просто индекс буквы (например, число 14)
        
        # Шаг А: Превращаем индекс в плотный вектор
        # .view(1, 1, -1) — это технический момент PyTorch.
        # Нейросеть ждет 3D-массив: [Batch Size, Sequence Length, Features]
        # Мы говорим: "У нас 1 пакет, 1 буква в цепочке, и авто-размер вектора (-1)"
        embedded = self.embedding(input).view(1, 1, -1)
        
        # Шаг Б: Прогоняем через GRU
        output, hidden = self.gru(embedded, hidden)
        
        return output, hidden

    # Вспомогательный метод для создания пустой "памяти" (нулей) перед началом чтения
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [6]:
# 1. Создаем экземпляр Энкодера
# vocab_size мы получили на прошлом этапе (около 66)
# hidden_size = 128 (размер нашего "мозга")
test_encoder = EncoderRNN(vocab_size, 128).to(device)

# 2. Эмулируем вход: берем индекс буквы 'A'
# tensor([[14]]) — это тензор, содержащий индекс.
# .to(device) отправляет его на видеокарту.
test_input = torch.tensor([[char2idx['A']]], device=device)

# 3. Эмулируем пустую память
test_hidden = test_encoder.initHidden()

# 4. Прогоняем (Forward pass)
test_output, test_hidden_new = test_encoder(test_input, test_hidden)

print(f"Размер входа: {test_input.shape}") # Должен быть [1, 1]
assert test_input.shape == torch.Size([1,1])



print(f"Размер выхода (памяти): {test_hidden_new.shape}") # Должен быть [1, 1, 128]

assert test_hidden_new.shape == torch.Size([1, 1, 128])



print("✅ Энкодер работает корректно!")

Размер входа: torch.Size([1, 1])
Размер выхода (памяти): torch.Size([1, 1, 128])
✅ Энкодер работает корректно!


In [7]:
import torch.nn.functional as F

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, max_length=15):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.max_length = max_length # Максимальная длина фразы (ограничение матрицы внимания)

        # 1. Эмбеддинг для выходных символов (Английские буквы/цифры)
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)

        # 2. Слой Внимания (Самый важный!)
        # Вход: (Текущий эмеддинг + Скрытое состояние) -> Размер = hidden_size * 2
        # Выход: Коэффициенты важности для каждого из max_length шагов
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)

        # 3. Слой Смешивания (Combine)
        # Объединяет (Текущий эмбеддинг + Результат внимания) -> hidden_size
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)

        # 4. GRU (обычная память)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)

        # 5. Выходной слой (Проекция в словарь)
        # Превращает вектор мысли (128) в вероятности букв (66)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        # input: Индекс одной буквы (1, 1)
        # hidden: Память с прошлого шага (1, 1, 128)
        # encoder_outputs: Все векторы прочитанной фразы (max_length, 128)

        embedded = self.embedding(input).view(1, 1, -1) # (1, 1, 128)

        # ШАГ А: Вычисляем Attention Weights
        # Соединяем (concat) вход и память: [1, 256]
        attn_input = torch.cat((embedded[0], hidden[0]), 1)
        
        # Прогоняем через линейный слой и Softmax -> получаем проценты
        # Результат: [1, 15] (например: 0.1, 0.8, 0.1 ...)
        attn_weights = F.softmax(self.attn(attn_input), dim=1)

        # ШАГ Б: Применяем внимание (Матричное умножение)
        # (1, 1, 15) x (1, 15, 128) -> (1, 1, 128)
        # Мы "взвесили" выходы энкодера.
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        # ШАГ В: Скармливаем это в GRU
        # Соединяем то, что пришло на вход, с тем, что нашли вниманием
        output = torch.cat((embedded[0], attn_applied[0]), 1) # (1, 256)
        output = self.attn_combine(output).unsqueeze(0)       # (1, 1, 128)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        # ШАГ Г: Предсказание следующей буквы
        output = F.log_softmax(self.out(output[0]), dim=1)

        # Возвращаем attn_weights, чтобы потом их нарисовать!
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [8]:
# 1. Создаем Декодер
test_decoder = AttnDecoderRNN(hidden_size=128, output_size=vocab_size).to(device)

# 2. Подготовим фейковые данные (эмуляция ситуации)
# Вход: Индекс символа '<' (начало строки)
test_input = torch.tensor([[char2idx['<']]], device=device)
# Память: берем из теста энкодера (или нули)
test_hidden = test_encoder.initHidden() # (1, 1, 128)
# Выходы энкодера: Эмулируем, будто энкодер прочитал фразу из 15 букв
# Создаем матрицу (15, 128)
test_encoder_outputs = torch.zeros(15, 128, device=device)

# 3. Прогоняем (Forward pass)
test_output, test_hidden_new, test_attn = test_decoder(test_input, test_hidden, test_encoder_outputs)

print(f"Размер предсказания (вероятности букв): {test_output.shape}") # (1, 66)
assert test_output.shape == torch.Size([1,66])


print(f"Размер весов внимания: {test_attn.shape}") # (1, 15) - важно!
print("✅ Декодер работает корректно!")
assert test_attn.shape == torch.Size([1,15])



Размер предсказания (вероятности букв): torch.Size([1, 66])
Размер весов внимания: torch.Size([1, 15])
✅ Декодер работает корректно!


In [9]:
# Маркер конца строки (End Of Sentence).
# В нашем генераторе мы оборачиваем дату в <...>.
# Значит, символом остановки будет '>'.
EOS_token = char2idx['>']

def tensorFromSentence(sentence):
    # 1. Превращаем строку в список индексов
    indexes = [char2idx[char] for char in sentence]
    
    # 2. Добавляем маркер конца (опционально, но полезно для декодера)
    # В нашем случае '>' уже есть в конце строки от генератора, 
    # но для надежности можно убедиться, что он там есть.
    # Наш генератор выдает '<...>', так что '>' уже внутри indexes.
    
    # 3. Превращаем в Тензор
    # dtype=torch.long обязателен для индексов (Embedding слой требует long)
    # .view(-1, 1) делает из вектора столбик (Sequence Length x Batch Size)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

In [10]:
# teacher_forcing_ratio = 0.5: 
# В 50% случаев мы сами подсказываем правильный ответ, 
# в 50% - заставляем модель опираться на свои предсказания.
teacher_forcing_ratio = 0.5

def train_step(input_tensor, target_tensor, encoder, decoder, 
               encoder_optimizer, decoder_optimizer, criterion):
    
    # 1. Инициализация
    encoder_hidden = encoder.initHidden()

    # Сбрасываем накопленные градиенты
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # Массив для хранения всех выходов энкодера
    encoder_outputs = torch.zeros(decoder.max_length, encoder.hidden_size, device=device)

    # === ВАЖНО: Эта строка должна быть ЗДЕСЬ, с таким же отступом, как for ниже ===
    # Создаем нулевой тензор. Это база для накопления ошибки.
    loss = torch.tensor(0.0, device=device)

    # === 2. ЗАПУСК ЭНКОДЕРА ===
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    # === 3. ЗАПУСК ДЕКОДЕРА ===
    decoder_input = torch.tensor([[char2idx['<']]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Режим Учителя
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di] 
    else:
        # Режим Самостоятельности
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.detach()
            if decoder_input.item() == char2idx['>']:
                break

    # === 4. ОБУЧЕНИЕ ===
    # Pylance ругался здесь, потому что боялся, что loss не определен.
    # Но теперь он точно определен в начале функции.
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [11]:
# Настройки
TOTAL_SAMPLES = 10000  # Сколько всего примеров хотим
TRAIN_RATIO = 0.8      # 80% на обучение

# 1. Генерируем уникальный пул данных
# Используем set, чтобы исключить дубликаты (одинаковые даты)
dataset_set = set()

print(f"🔄 Генерируем {TOTAL_SAMPLES} уникальных пар...")

while len(dataset_set) < TOTAL_SAMPLES:
    # Вызываем твою функцию (она должна быть уже определена выше)
    pair = generate_date() 
    dataset_set.add(pair)

# Превращаем в список и перемешиваем
full_dataset = list(dataset_set)
random.shuffle(full_dataset)

# 2. Разрезаем на Train и Test
split_index = int(TOTAL_SAMPLES * TRAIN_RATIO)

train_data = full_dataset[:split_index]
test_data = full_dataset[split_index:]

print(f"✅ Готово!")
print(f"📚 Обучающая выборка (Train): {len(train_data)} шт.")
print(f"🎓 Тестовая выборка (Test):  {len(test_data)} шт. (Модель их никогда не увидит)")
print(f"Пример из теста: {test_data[0]}")

🔄 Генерируем 10000 уникальных пар...
✅ Готово!
📚 Обучающая выборка (Train): 8000 шт.
🎓 Тестовая выборка (Test):  2000 шт. (Модель их никогда не увидит)
Пример из теста: ('12 Nov 1965', '<1965-11-12>')


In [ ]:
import time
import matplotlib.pyplot as plt
import torch.optim as optim

# Настройки обучения
hidden_size = 128
n_iters = 20000  # Количество шагов обучения
learning_rate = 0.01

# Инициализируем модели заново (чтобы стереть старую память/веса)
encoder = EncoderRNN(vocab_size, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, vocab_size).to(device)

encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

plot_losses = []
print_every = 1000
current_loss = 0
all_losses = []

print(f"🚀 Начинаем ЧЕСТНОЕ обучение на {len(train_data)} примерах...")
start = time.time()

for iter in range(1, n_iters + 1):
    # ВАЖНОЕ ИЗМЕНЕНИЕ:
    # Берем случайную пару ТОЛЬКО из train_data
    # (random.choice выбирает один случайный элемент из списка)
    training_pair = random.choice(train_data)
    src_str = training_pair[0]
    tgt_str = training_pair[1]
    
    # Дальше всё как раньше
    input_tensor = tensorFromSentence(src_str)
    target_tensor = tensorFromSentence(tgt_str)

    loss = train_step(input_tensor, target_tensor, encoder, decoder,
                     encoder_optimizer, decoder_optimizer, criterion)
    
    current_loss += loss
    all_losses.append(loss)

    if iter % print_every == 0:
        print_loss_avg = current_loss / print_every
        current_loss = 0
        print(f'{iter} ({iter / n_iters * 100:.0f}%) time: {time.time()-start:.2f}s | loss: {print_loss_avg:.4f}')

print("✅ Обучение завершено.")

🚀 Начинаем ЧЕСТНОЕ обучение на 8000 примерах...


In [ ]:
import numpy as np

# Функция сглаживания (свертка)
def moving_average(data, window_size=50):
    # window_size=50 означает, что каждая точка графика - это среднее за 50 шагов
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

# Применяем фильтр к твоему списку ошибок
smooth_losses = moving_average(all_losses)

# Рисуем
plt.figure(figsize=(10, 5))
plt.plot(smooth_losses, label='Smoothed Loss')
plt.title("График обучения (Скользящее среднее)")
plt.xlabel("Итерации")
plt.ylabel("Ошибка")
plt.grid(True, alpha=0.3) # Добавим сетку для наглядности
plt.legend()
plt.show()

In [ ]:
def evaluate(input_sentence):
    with torch.no_grad(): # Отключаем градиенты для теста
        input_tensor = tensorFromSentence(input_sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        # Массив для хранения весов внимания (для картинки)
        decoder_attentions = torch.zeros(decoder.max_length, decoder.max_length)

        encoder_outputs = torch.zeros(decoder.max_length, encoder.hidden_size, device=device)

        # 1. Читаем (Encoder)
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        # 2. Пишем (Decoder)
        decoder_input = torch.tensor([[char2idx['<']]], device=device)
        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(decoder.max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            # Сохраняем веса внимания
            decoder_attentions[di] = decoder_attention.data
            
            topv, topi = decoder_output.data.topk(1)
            
            if topi.item() == char2idx['>']:
                break # Конец фразы
            else:
                decoded_words.append(idx2char[topi.item()])

            decoder_input = topi.detach()

        return "".join(decoded_words), decoder_attentions[:di + 1]

In [ ]:
def evaluate_randomly(n=5):
    print(f"\n🎓 ЭКЗАМЕН (Данные из Test Set):")
    correct = 0
    
    for i in range(n):
        # Берем из TEST data
        pair = random.choice(test_data)
        src = pair[0]
        tgt_real = pair[1]
        
        # Просим модель предсказать
        output_words, _ = evaluate(src) # Используем нашу функцию evaluate
        
        # Сравниваем (убираем маркеры < > для красоты)
        tgt_clean = tgt_real.replace('<','').replace('>','')
        out_clean = output_words.replace('<','').replace('>','')
        
        # Простая проверка на точное совпадение
        is_match = "✅" if tgt_clean == out_clean else f"❌ (Ожидалось: {tgt_clean})"
        if tgt_clean == out_clean: correct += 1
            
        print(f"Вход: {src.ljust(15)} -> Выход: {out_clean} {is_match}")
        
    print(f"\nТочность на выборке: {correct}/{n}")

# Запускаем экзамен
evaluate_randomly(10)

In [ ]:
import matplotlib.ticker as ticker

def showAttention(input_sentence):
    output_words, attentions = evaluate(input_sentence)
    
    # Рисуем тепловую карту
    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Подписи осей
    ax.set_xticklabels([''] + list(input_sentence), rotation=90)
    ax.set_yticklabels([''] + list(output_words))
    
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    print(f"Вход:  {input_sentence}")
    print(f"Выход: {output_words}")

# === ЗАПУСК ПРОВЕРКИ ===
# Проверяем на датах, которых модель НЕ видела
def showAttentions(n = 3):
    for i in range(n):
        pair = random.choice(test_data)
        src = pair[0]
        showAttention(src)
